* Understand importance of cross validation
* How to perform cross validation using sklearn library
* Understand various parameters involved in cross validation

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('insurance.csv')
data.head()

age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520

In [3]:
ohe = pd.get_dummies(data)
y = 'charges'
x = ohe.columns.drop(y)
ohe.head()

age     bmi  children      charges  sex_female  sex_male  smoker_no  \
0   19  27.900         0  16884.92400           1         0          0   
1   18  33.770         1   1725.55230           0         1          1   
2   28  33.000         3   4449.46200           0         1          1   
3   33  22.705         0  21984.47061           0         1          1   
4   32  28.880         0   3866.85520           0         1          1   

   smoker_yes  region_northeast  region_northwest  region_southeast  \
0           1                 0                 0                 0   
1           0                 0                 0                 1   
2           0                 0                 0                 1   
3           0                 0                 1                 0   
4           0                 0                 1                 0   

   region_southwest  
0                 1  
1                 0  
2                 0  
3                 0  
4                 0

In [8]:
xtrain, xtest, ytrain, ytest = train_test_split(ohe[x], ohe[y], test_size=.2, random_state=100)
#try with different random states say 10 and 100

In [9]:
dr = DecisionTreeRegressor(max_depth=5).fit(xtrain, ytrain)
ypred_test = dr.predict(xtest)

print('RMSE: %.2f' % np.sqrt(mean_squared_error(ytest, ypred_test)))
print('R2 Score : %.2f' % r2_score(ytest, ypred_test))

RMSE: 4695.77
R2 Score : 0.86


In [ ]:
RMSE: 5053.92
R2 Score : 0.82

# Cross validation - calc score

In [10]:
dr = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(dr, xtrain, ytrain, cv=5)
scores

array([0.75353786, 0.82228457, 0.82251922, 0.82882194, 0.86335774])

* Above score is R2_score

In [11]:
avg_score = scores.mean()
sqr_std = scores.std()*2
print(avg_score)
print('Expected R Square: %.2f (+/- %.2f)' %(avg_score, sqr_std))
print('Range for R Square: %.2f - %.2f' %(avg_score - sqr_std, avg_score + sqr_std))

0.8181042664771079
Expected R Square: 0.82 (+/- 0.07)
Range for R Square: 0.75 - 0.89


# V.Imp - Scorings with CV

In [12]:
scores = cross_val_score(dr, xtrain, ytrain, cv=5, scoring='neg_root_mean_squared_error')
scores

array([-5157.63405658, -5069.92445793, -5667.04277152, -4926.90782911,
       -4293.20629115])

In [13]:
cv_results = cross_validate(dr, xtrain, ytrain, cv=5)
cv_results # returns time it took to calc the scores

{'fit_time': array([0.00351405, 0.00314617, 0.00295925, 0.0059526 , 0.00335622]),
 'score_time': array([0.00181127, 0.00163007, 0.00225449, 0.00278091, 0.00180531]),
 'test_score': array([0.75353786, 0.82228457, 0.82065872, 0.82882194, 0.86335774])}

# Display train scores also

In [14]:
cv_results = cross_validate(dr, xtrain, ytrain, cv=5, return_train_score=True)
cv_results # returns time it took to calc the scores

{'fit_time': array([0.00573158, 0.00409746, 0.003613  , 0.00472212, 0.00489092]),
 'score_time': array([0.00278544, 0.00210428, 0.00226045, 0.00380516, 0.00121427]),
 'test_score': array([0.76458247, 0.82228457, 0.82065872, 0.82882194, 0.86335774]),
 'train_score': array([0.88467818, 0.87973756, 0.88438821, 0.87540047, 0.87123303])}

## Use list of multiple scoring parameters

In [15]:
cv_results = cross_validate(dr, xtrain, ytrain, cv=5, return_train_score=True, scoring=['neg_root_mean_squared_error', 'r2'])
cv_results # returns time it took to calc the scores

{'fit_time': array([0.00580812, 0.00319552, 0.00263929, 0.00388384, 0.00433779]),
 'score_time': array([0.00305438, 0.00177646, 0.00184774, 0.00290632, 0.00290632]),
 'test_neg_root_mean_squared_error': array([-5157.63405658, -5069.92445793, -5696.66867934, -4926.90782911,
        -4293.20629115]),
 'train_neg_root_mean_squared_error': array([-4175.95303209, -4152.86444037, -3935.14728426, -4238.9702217 ,
        -4333.00638746]),
 'test_r2': array([0.76458247, 0.82228457, 0.82065872, 0.82882194, 0.86335774]),
 'train_r2': array([0.88467818, 0.87973756, 0.88438821, 0.87540047, 0.87123303])}

In [16]:
cv_df = pd.DataFrame(cv_results) # converting results into df
cv_df.head()

fit_time  score_time  test_neg_root_mean_squared_error  \
0  0.005808    0.003054                      -5157.634057   
1  0.003196    0.001776                      -5069.924458   
2  0.002639    0.001848                      -5696.668679   
3  0.003884    0.002906                      -4926.907829   
4  0.004338    0.002906                      -4293.206291   

   train_neg_root_mean_squared_error   test_r2  train_r2  
0                       -4175.953032  0.764582  0.884678  
1                       -4152.864440  0.822285  0.879738  
2                       -3935.147284  0.820659  0.884388  
3                       -4238.970222  0.828822  0.875400  
4                       -4333.006387  0.863358  0.871233